Домашка основана на этом семинаре - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Embeddings.ipynb

Возьмите данные соревнования по определению перефразирования - http://paraphraser.ru/download/get?file_id=1

Задание делится на 2 части:

1) Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (любой). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 
Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 
ВАЖНО: Оценивать модели нужно с помощью кросс-валидации! Метрика - f1.

2) Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).  Попробуйте улучшить метрику, изменив параметры в методах векторизации.


SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части). 


Оценивание - если вы сделали всё вышеперечисленное - 10 баллов. Каждая ошибка - минус 0.5 балла. 

Выложите код к себе на гитхаб и вставьте ссылку в поле ниже (в тетрадке должны быть показатели метрик и ваши комментарии).

In [1]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter, defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from scipy import spatial

import warnings
warnings.filterwarnings('ignore')

# Задание 1

In [2]:
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


Загрузка корпуса

In [3]:
with open('corpus_hum.txt', encoding='utf-8') as file:
    data = file.read().splitlines()


In [4]:
# data_norm = [normalize(text) for text in data]

In [5]:
# data_norm = [text for text in data_norm if text]

In [6]:
# with open('norm_corp', 'w', encoding='utf-8') as file:
#     for line in data_norm:
#         file.write(line + '\n')

In [7]:
with open('norm_corp', encoding='utf-8') as file:
    data_norm = file.readlines()

Теги НКРЯ

In [8]:
from pymystem3 import Mystem
mystem = Mystem()

mapping = {}

# mystem POS tags - > RNC POS tags
for line in open('ru-rnc.map.txt'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud

Загрузка и обучение моделей

In [9]:
# w2model = gensim.models.Word2Vec([text.split() for text in data_norm], size=30)

In [10]:
# w2model.save('w2model.model')

In [11]:
w2model = gensim.models.Word2Vec.load('w2model.model')

In [12]:
# ruscorpora_upos_cbow_300_20_2019
rv_w2model = gensim.models.KeyedVectors.load_word2vec_format('182/model.bin', binary=True)

Функции, необходимые для создания эмбедингов

In [13]:
#готовые модели w2v требуют POS тегов, поэтому требуется их отдельно проставлять

def pos_tag_word(word, mst_analyzer, pos_map):
    
    mst_word = mst_analyzer.analyze(word)
    try:
        mst_word = mst_word[0]['analysis'][0]
        lemma = mst_word['lex'].lower().strip()
        pos = mst_word['gr'].split(',')[0]
        pos = mapping[pos]
        
    except:
        return word
    
    return f'{lemma}_{pos}' 

In [14]:
def vec_from_model(word, pos_map, model, dim=300, pos_tag=False, mst_analyzer=None, cache=None, verbose=False):
    if pos_tag:
        if not mst_analyzer:
            raise ValueError('A morphological analyzer is required with pos_tag set to True')
        
        # кэш для ускорения скорости работы функции
        if cache is not None:
            if word in cache:
                word = cache[word]
            else:
                cache[word] = pos_tag_word(word, mst_analyzer, pos_map)
                word = cache[word]
        else:
            word = pos_tag_word(word, mst_analyzer, pos_map)
        
    
    vector = []
    try:
        vector = model[word]
        
    except:
        # на случай, если модели не удалось выдать данное слово
        vector = np.zeros(dim)
        if verbose:
            print(f'Word: {word} not found in the model.')
    
    return vector

In [15]:
pos_tag_word('язык', mystem, mapping)

'язык_NOUN'

In [16]:
word_cache = {}

In [17]:
test_manvec = vec_from_model('язык', mapping, dim=30, model=w2model)

In [18]:
test_rusvec =  vec_from_model('язык', mapping, dim=300, model=rv_w2model,
                              pos_tag=True, mst_analyzer=mystem, cache=word_cache)

In [19]:
word_cache

{'язык': 'язык_NOUN'}

In [20]:
test_rusvec.shape, test_manvec.shape

((300,), (30,))

In [21]:
# non-zero vectors were returned by the model
test_rusvec.any(), test_manvec.any()

(True, True)

In [22]:
def text_embedding(text, pos_map, model, dim=300, pos_tag=False, mst_analyzer=None, cache=None, verbose=False):
    
    text = text.split()
    words = Counter(text)
    total = len(text)
    vector_models = np.zeros((len(text), dim))
    
    for i, word in enumerate(words):
        word_vec = vec_from_model(word, model=model, pos_map=pos_map,
                                  dim=dim, pos_tag=pos_tag, mst_analyzer=mst_analyzer,
                                  cache=cache, verbose=verbose)
        vector_models[i] = word_vec * (words[word]/total)
                
    if vector_models.any():
        vector_model = np.average(vector_models, axis=0)
    else:
        vector_model = np.zeros((dim))
    
    return vector_model
        


Проверяю, что все работает как надо

In [23]:
some_test = text_embedding('Какой-то текст пример.', mapping, model=rv_w2model, dim=300, pos_tag=True, mst_analyzer=mystem, verbose=False)

In [24]:
some_test.shape, some_test.any()

((300,), True)

In [25]:
other_test = text_embedding('Какой-то текст пример.', mapping, model=w2model, dim=30)

In [26]:
other_test.shape, other_test.any()

((30,), True)

Загрузка перифраз

In [27]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
para = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [28]:
para['text_1_norm'] = para['text_1'].apply(normalize)
para['text_2_norm'] = para['text_2'].apply(normalize)

In [29]:
X_text_1 = [text_embedding(text, mapping, model = rv_w2model,
            dim=300, pos_tag=True, mst_analyzer=mystem, cache=word_cache) for text
           in para['text_1_norm']]
X_text_2 = [text_embedding(text, mapping, model = rv_w2model,
            dim=300, pos_tag=True, mst_analyzer=mystem, cache=word_cache) for text
           in para['text_2_norm']]

X_text = np.concatenate([X_text_1, X_text_2], axis=1)

In [30]:
X_text.shape

(7227, 600)

In [31]:
X_text_1_cust = [text_embedding(text, mapping, model = w2model,
            dim=30) for text
           in para['text_1_norm']]
X_text_2_cust = [text_embedding(text, mapping, model = w2model,
            dim=30) for text
           in para['text_2_norm']]

X_text_cust = np.concatenate([X_text_1_cust, X_text_2_cust], axis=1)

In [32]:
X_text_cust.shape

(7227, 60)

In [33]:
y = para.label.values

In [34]:
clf = LogisticRegression(C=1000, class_weight='balanced')

In [35]:
cv_scores_1 = cross_val_score(clf, X_text, y, scoring='f1_micro', cv=5)
print('Rusvectores model results:')
print(f'CV score values: {cv_scores_1}')
print(f'Average F-score: {np.mean(cv_scores_1)}')

Rusvectores model results:
CV score values: [0.39737388 0.40774015 0.44705882 0.35249307 0.36565097]
Average F-score: 0.39406337937526237


In [36]:
cv_scores_2 = cross_val_score(clf, X_text_cust, y, scoring='f1_micro', cv=5)
print('Custom w2v model results:')
print(f'CV score values: {cv_scores_2}')
print(f'Average F-score: {np.mean(cv_scores_2)}')

Custom w2v model results:
CV score values: [0.42225294 0.44298549 0.47058824 0.39819945 0.3767313 ]
Average F-score: 0.42215148150854953


Видим, что готовая модель не настолько уж сильно лучше. Значительное преимущество в объеме и количестве признаков дает непропорциональный прирост в качестве. Предположу, что если хорошо подобрать тексты, то своя модель будет работать даже лучше готовой, имея при этом меньшее количество фич.

# Задание 2

## Word2Vec

In [37]:
# костыльное решение, но я пока разобрался как это нормально сделать в пандасе
w2v_sim = []
for x, y in zip(X_text_1, X_text_2):
    if x.any() and y.any():
        w2v_sim.append(1 - spatial.distance.cosine(x, y))
    else:
        w2v_sim.append(0)
w2v_sim = np.array(w2v_sim)

In [38]:
para['w2v_sim'] = w2v_sim

In [39]:
# правильно было бы положить это в функцию, но пока не стал, чтобы не путаться
w2v_sim_cus = []
for x, y in zip(X_text_1_cust, X_text_2_cust):
    if x.any() and y.any():
        w2v_sim_cus.append(1 - spatial.distance.cosine(x, y))
    else:
        w2v_sim_cus.append(0)
w2v_sim_cus = np.array(w2v_sim_cus)

In [40]:
para ['w2v_sim_cus'] = w2v_sim_cus

In [41]:
clf = LogisticRegression()

In [42]:
cv_sim = cross_val_score(clf, w2v_sim.reshape(-1, 1), para.label.values, scoring='f1_micro', cv=5)
print('Rusvectores model results:')
print(f'CV score values: {cv_sim}')
print(f'Average F-score: {np.mean(cv_sim)}')

Rusvectores model results:
CV score values: [0.57014513 0.58396683 0.59238754 0.45844875 0.49792244]
Average F-score: 0.5405741380317778


Кажется, что самостоятельно обученная модель не намного хуже

In [43]:
cv_cus = cross_val_score(clf, w2v_sim_cus.reshape(-1, 1), para.label.values, scoring='f1_micro', cv=5)
print('Custom w2v model results:')
print(f'CV score values: {cv_cus}')
print(f'Average F-score: {np.mean(cv_cus)}')

Custom w2v model results:
CV score values: [0.53489979 0.54872149 0.55847751 0.44736842 0.45498615]
Average F-score: 0.5088906729411489


## Fasttext

In [44]:
ft_model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [45]:
ft_1 = ft_model[para['text_1']]
ft_2 = ft_model[para['text_2']]

In [46]:
ft_sim = []
for x, y in zip(ft_1, ft_2):
    if x.any() and y.any():
        ft_sim.append(1 - spatial.distance.cosine(x, y))
    else:
        ft_sim.append(0)
ft_sim = np.array(ft_sim)

In [60]:
para['ft_sim'] = ft_sim

In [47]:
cv_ft = cross_val_score(clf, ft_sim.reshape(-1, 1), para.label.values, scoring='f1_micro', cv=5)
print('Fasttext model results:')
print(f'CV score values: {cv_ft}')
print(f'Average F-score: {np.mean(cv_ft)}')

Fasttext model results:
CV score values: [0.55079475 0.57567381 0.5799308  0.49445983 0.50761773]
Average F-score: 0.5416953827613927


Любопытно, что использование нормализованных текстов все же повышает качество.

In [48]:
ft_1_norm = ft_model[para['text_1_norm']]
ft_2_norm = ft_model[para['text_2_norm']]

In [49]:
ft_sim_norm = []
for x, y in zip(ft_1_norm, ft_2_norm):
    if x.any() and y.any():
        ft_sim_norm.append(1 - spatial.distance.cosine(x, y))
    else:
        ft_sim_norm.append(0)
ft_sim_norm = np.array(ft_sim_norm)

In [61]:
para['ft_sim_norm'] = ft_sim_norm

In [50]:
cv_ft_norm = cross_val_score(clf, ft_sim_norm.reshape(-1, 1), para.label.values, scoring='f1_micro', cv=5)
print('Fasttext model results:')
print(f'CV score values: {cv_ft_norm}')
print(f'Average F-score: {np.mean(cv_ft_norm)}')

Fasttext model results:
CV score values: [0.56323428 0.57774706 0.60069204 0.51246537 0.49376731]
Average F-score: 0.5495812138416014


## SVD

Пробовал добавлять ngram_range, но ничего не дало. Остальные параметры подкрутил до максимального качества.

In [51]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.7, max_features=1200)
tfidf.fit(pd.concat([para['text_1_norm'], para['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.7, max_features=1200,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Увеличение количества компонентов и количества итераций дает небольшой приросто в качестве. Свыше 500 прироста нет.

In [52]:
svd = TruncatedSVD(500, n_iter=20, random_state=42)

svd.fit(tfidf.transform(pd.concat([para['text_1_norm'], para['text_2_norm']])))

TruncatedSVD(algorithm='randomized', n_components=500, n_iter=20,
             random_state=42, tol=0.0)

In [53]:
X_text_1_svd = svd.transform(tfidf.transform(para['text_1_norm']))
X_text_2_svd = svd.transform(tfidf.transform(para['text_2_norm']))

X_text_svd = np.concatenate([X_text_1_svd, X_text_2_svd], axis=1)

In [54]:
svd_sim = []
for x, y in zip(X_text_1_svd, X_text_2_svd):
    if x.any() and y.any():
        svd_sim.append(1 - spatial.distance.cosine(x, y))
    else:
        svd_sim.append(0)
svd_sim = np.array(svd_sim)

In [63]:
para['svd_sim'] = svd_sim

In [55]:
cv_svd = cross_val_score(clf, svd_sim.reshape(-1, 1), para.label.values, scoring='f1_micro', cv=5)
print('SVD results:')
print(f'CV score values: {cv_svd}')
print(f'Average F-score: {np.mean(cv_svd)}')

SVD results:
CV score values: [0.55148583 0.57774706 0.57716263 0.44736842 0.44598338]
Average F-score: 0.5199494651915932


## NMF

Использование параметра init повышает скорость обучения и качество. Другие параметры также несколько повысили качество.

In [56]:
nmf = NMF(50, init='nndsvd', random_state=42, alpha=0.2, tol=1e-1, solver='mu')
nmf.fit(tfidf.transform(pd.concat([para['text_1_norm'], para['text_2_norm']])))

NMF(alpha=0.2, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=50, random_state=42, shuffle=False, solver='mu', tol=0.1,
    verbose=0)

In [57]:
X_text_1_nmf = nmf.transform(tfidf.transform(para['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(para['text_2_norm']))

In [58]:
nmf_sim = []
for x, y in zip(X_text_1_nmf, X_text_2_nmf):
    if x.any() and y.any():
        nmf_sim.append(1 - spatial.distance.cosine(x, y))
    else:
        nmf_sim.append(0)
nmf_sim = np.array(nmf_sim)

In [64]:
para['nmf_sim'] = nmf_sim

In [59]:
cv_nmf = cross_val_score(clf, nmf_sim.reshape(-1, 1), para.label.values, scoring='f1_micro', cv=5)
print('NMF results:')
print(f'CV score values: {cv_nmf}')
print(f'Average F-score: {np.mean(cv_nmf)}')

NMF results:
CV score values: [0.51485833 0.54941258 0.53979239 0.41897507 0.38504155]
Average F-score: 0.48161598267264444


Резюмирую, что самой удобной моделью из рассмотренных можно считать Fasttext, т.к. она не требует нормализации текстов (хотя она может быть полезна) и при этом даёт результат *лучше*, чем Word2Vec. В целом её проще использовать

Модель, обученная на всех векторных близостях одновременно

In [68]:
feats = para[['w2v_sim', 'w2v_sim_cus', 'ft_sim', 'ft_sim_norm', 'svd_sim', 'nmf_sim']]

In [70]:
all_models = cross_val_score(clf, feats, para.label.values, scoring='f1_micro', cv=5)
print('All models result:')
print(f'CV score values: {all_models}')
print(f'Average F-score: {np.mean(all_models)}')

All models result:
CV score values: [0.58811334 0.6081548  0.61384083 0.51800554 0.51246537]
Average F-score: 0.5681159771117184
